In [1]:
from dataclasses import dataclass, asdict
import requests
import time
import duckdb
import pandas as pd
import numpy as np
from configparser import ConfigParser

%load_ext sql
%config SqlMagic.displaylimit = 10
%config SqlMagic.autopandas = True

In [2]:
config = ConfigParser()
config.read("../../.config")

['../../.config']

In [3]:
api_key = config["GoogleMaps"]["API_KEY"]
base_url = config["GoogleMaps"]["base_url"]
base_url_new = config["GoogleMaps"]["base_url_new"]
radius = 50000  # in meters
place_type = "shopping"  # Example place type
address = "Luxembourg City, Luxembourg"

In [4]:
# Connect to DuckDB
# con = duckdb.connect('places.db')
# df = con.execute('SELECT * FROM places').fetchdf()

In [5]:
conn = duckdb.connect("../../data/raw/database.db")
%sql conn --alias duckdb

In [6]:
%%sql
select column_name from  information_schema.columns
where table_name = 'places';

Running query in 'duckdb'

,column_name
0,place_id
1,name
2,vicinity
3,latitude
4,longitude


In [ ]:
%%sql
select count(1) from geonames

In [ ]:
%%sql

show tables;

In [ ]:
%%sql
select * from place_details

In [ ]:
# conn.close()

In [ ]:
%%sql

LAT_LONG << select latitude, longitude from geonames;

In [ ]:
def format_coordinates(latitude: float, longitude: float) -> str:
    return f"{latitude},{longitude}"

In [ ]:
location = format_coordinates(LAT_LONG.iloc[6,0], LAT_LONG.iloc[6,1])

In [ ]:
url = f"{base_url}/place/nearbysearch/json?location={location}&radius={radius}&key={api_key}&type=supermarket"

In [ ]:
response = requests.get(url)
places = response.json()
places['results']

In [ ]:
places

In [ ]:
place_id = 'ChIJgX9bQz236kcR5NpImkZBpoc'
details_url = f"{base_url}/place/details/json?place_id={place_id}&key={api_key}"
details_response = requests.get(details_url)


In [ ]:
details_response.json()['result']

#### Places API v2

In [16]:
# latitude = float(LAT_LONG.iloc[6,0])
# longitude = float(LAT_LONG.iloc[6,1])

latitude, longitude = 49.646692697148595, 6.12830393740609

radius = 5000

In [17]:
longitude,latitude

(6.045933, 49.6465214)

In [36]:
# Prepare the request payload
payload = {
"includedTypes": [
                #     'clothing_store',
                #   'discount_store',
                #     'gift_shop',
                #     'grocery_store',
                    'home_improvement_store',
                    # 'market',
                    # 'sporting_goods_store',
                    # 'wholesaler'
                    ],
"locationRestriction": {
    "circle": {
        "center": {
            "latitude": latitude,
            "longitude": longitude
        },
        "radius": radius
    }
},
"maxResultCount": 20,
#"rankPreference": "DISTANCE",
}

# Prepare the headers
headers = {
"Content-Type": "application/json",
"X-Goog-Api-Key": api_key,
"X-Goog-FieldMask": (
        "places.businessStatus,places.displayName,places.formattedAddress,"
        "places.googleMapsUri,places.id,places.location,places.plusCode,"
        "places.primaryType,places.types,places.internationalPhoneNumber,"
        "places.nationalPhoneNumber,places.priceLevel,places.rating,"
        "places.userRatingCount,places.websiteUri,places.delivery,places.dineIn,"
        "places.parkingOptions,places.paymentOptions,places.outdoorSeating,"
        "places.reservable,places.restroom"
    )
}

In [37]:
response = requests.post(f'{base_url_new}:searchNearby', json=payload, headers=headers)

In [38]:
response.json()

{'places': [{'id': 'ChIJebfOoPNLlUcR769qUUOkfI8',
   'types': ['home_improvement_store',
    'home_goods_store',
    'point_of_interest',
    'store',
    'establishment'],
   'nationalPhoneNumber': '31 66 55 1',
   'internationalPhoneNumber': '+352 31 66 55 1',
   'formattedAddress': '43 Rue du puits Romain, 8070 Bertrange, Luxembourg',
   'plusCode': {'globalCode': '8FX8J35G+2W',
    'compoundCode': 'J35G+2W Bertrange, Luxembourg'},
   'location': {'latitude': 49.6075999, 'longitude': 6.0773133},
   'rating': 4.2,
   'googleMapsUri': 'https://maps.google.com/?cid=10339319453571526639',
   'websiteUri': 'https://www.hornbach.lu/mein-markt/baumarkt-hornbach-bertrange/',
   'businessStatus': 'OPERATIONAL',
   'userRatingCount': 5413,
   'displayName': {'text': 'Hornbach Bertrange', 'languageCode': 'en'},
   'delivery': True,
   'primaryType': 'home_improvement_store',
   'paymentOptions': {'acceptsCreditCards': True,
    'acceptsDebitCards': True,
    'acceptsCashOnly': False,
    'acce

In [39]:
response.json()["places"][0]

{'id': 'ChIJebfOoPNLlUcR769qUUOkfI8',
 'types': ['home_improvement_store',
  'home_goods_store',
  'point_of_interest',
  'store',
  'establishment'],
 'nationalPhoneNumber': '31 66 55 1',
 'internationalPhoneNumber': '+352 31 66 55 1',
 'formattedAddress': '43 Rue du puits Romain, 8070 Bertrange, Luxembourg',
 'plusCode': {'globalCode': '8FX8J35G+2W',
  'compoundCode': 'J35G+2W Bertrange, Luxembourg'},
 'location': {'latitude': 49.6075999, 'longitude': 6.0773133},
 'rating': 4.2,
 'googleMapsUri': 'https://maps.google.com/?cid=10339319453571526639',
 'websiteUri': 'https://www.hornbach.lu/mein-markt/baumarkt-hornbach-bertrange/',
 'businessStatus': 'OPERATIONAL',
 'userRatingCount': 5413,
 'displayName': {'text': 'Hornbach Bertrange', 'languageCode': 'en'},
 'delivery': True,
 'primaryType': 'home_improvement_store',
 'paymentOptions': {'acceptsCreditCards': True,
  'acceptsDebitCards': True,
  'acceptsCashOnly': False,
  'acceptsNfc': True}}

In [ ]:
place_id = 'ChIJo3eV1sFMlUcR5LsVjBSDAZA'
details_url = f"{base_url_new}/{place_id}"
headers = {
"Content-Type": "application/json",
"X-Goog-Api-Key": api_key,
"X-Goog-FieldMask": "*"  # Adjust fields as needed
}
details_response = requests.get(details_url, headers=headers)

In [ ]:
details_response.json()